In [3]:
import nltk

nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import tensorflow as tf

keras = tf.keras

In [5]:
%matplotlib inline
import re
import seaborn as sbn
import nltk
import tqdm as tqdm
import sqlite3
import pandas as pd
import numpy as np

import string
from nltk.corpus import stopwords
stop = stopwords.words("english")
import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from math import floor,ceil
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

from sklearn.svm import LinearSVC
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers import LSTM, Dense, Embedding
from keras.models import Model


from gensim import summarization
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('Reviews1.csv')

In [7]:
review_data = df[['UserId', 'ProductId', 'Score','Summary','Text']]
review_data.head()

,UserId,ProductId,Score,Summary,Text
0,A3SGXH7AUHU8GW,B001E4KFG0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,A1D87F6ZCVE5NK,B00813GRG4,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,ABXLMWJIXXAIN,B000LQOCH0,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,A395BORC6FGVXV,B000UA0QIQ,2,Cough Medicine,If you are looking for the secret ingredient i...
4,A1UQRSCLF8GW1T,B006K2ZZ7K,5,Great taffy,Great taffy at a great price. There was a wid...


In [8]:
review_data['UserId'].value_counts()

A3NHUQ33CFH3VM    5
A3PJZ8TU8FDQ1K    5
A2NLZ3M0OJV9NX    4
A3RMGIKUWGPZOK    4
A31N6KB160O508    3
                 ..
A19SDJ2TSSL8HI    1
A15SCIHLW004UA    1
A1QCP3UXNAN8FJ    1
A3RXAU2N8KV45G    1
A3NH41GQPFMGQS    1
Name: UserId, Length: 2890, dtype: int64

In [9]:

# user1_data = review_data.loc[review_data['UserId'] == 'A3NHUQ33CFH3VM']
labels = review_data['Score'].map(lambda x : 1 if int(x) > 3 else 0)
print(np.array(labels))

[1 0 1 ... 1 0 1]


In [10]:
review_data['Score'] = review_data['Score'].map(lambda x : 1 if int(x) > 3 else 0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
review_data['Score'].value_counts()

1    2297
0     702
Name: Score, dtype: int64

In [12]:
def data_clean(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    # text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    # text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text
# def data_clean(text):
#   #convert text to lower case
#   text = text.lower()

#   #remove numbers 
#   text = re.sub(r'\d+', '', text)
#   #remove punctuation
#   text = text.translate(str.maketrans("","", string.punctuation))

#   #remove white spaces
#   text = text.strip()

#   #stop words removal
#   stop_words = set(stopwords.words('english'))
#   tokens = word_tokenize(text)
#   result = [i for i in tokens if not i in stop_words]
#   text = " ".join(result)
#   # print(text)

#   text = text.split()
#   stemmer = SnowballStemmer('english')
#   stemmed_words = [stemmer.stem(word) for word in text]
#   text = " ".join(stemmed_words)

#   #lemmatization
#   lemmatizer=WordNetLemmatizer()
#   text=word_tokenize(text)

#   return text

In [13]:
# clean_reviewData = []
# for rev in review_data['Text']:
#     clean_reviewData.append( " ".join(data_clean(rev)))

# clean_summaryData = []
# for rev in review_data['Summary']:
#     clean_summaryData.append( " ".join(data_clean(rev)))

In [ ]:
def data_clean(rev, remove_stopwords=True):
  new_text = re.sub("[^a-zA-Z]"," ", rev)
  words = new_text.lower().split()
  if remove_stopwords:
        sts = set(stopwords.words("english"))
        words = [w for w in words if not w in sts]
  ary=[]
  eng_stemmer = english_stemmer 
  for word in words:
      ary.append(eng_stemmer.stem(word))  #stem the words to retain the root/base word
  return(ary)

In [22]:
Data = review_data[['Summary','Score']]
print(Data.head())
Data_x= Data['Summary']
Data_y= Data['Score']
#calculate padding length
MAX_LEN = 0
for i in range(1,2999):
  if(MAX_LEN < len(Data_x[i])):
    MAX_LEN = len(Data_x[i])
print(MAX_LEN)
pos=[]
neg=[]
for l in Data.Score:
  if l==0:
    pos.append(0)
    neg.append(1)
  elif l==1:
    pos.append(1)
    neg.append(0)

Data['Pos']=pos
Data['Neg']=neg

Data = Data[['Summary', 'Score', 'Pos', 'Neg']]
Data.head()

                 Summary  Score
0  Good Quality Dog Food      1
1      Not as Advertised      0
2  "Delight" says it all      1
3         Cough Medicine      0
4            Great taffy      1
126


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Summary,Score,Pos,Neg
0,Good Quality Dog Food,1,1,0
1,Not as Advertised,0,0,1
2,"""Delight"" says it all",1,1,0
3,Cough Medicine,0,0,1
4,Great taffy,1,1,0


In [23]:
Data['Text_clean'] = Data['Summary'].apply(lambda x: data_clean(x))
Data['Text_final'] = [' '.join(sen) for sen in Data['Text_clean']]
Data['Tokens'] = Data['Text_clean']

In [24]:
Data = Data[['Text_final', 'Tokens', 'Score', 'Pos', 'Neg']]
print(Data.head())

                                  Text_final                 Tokens  ...  Pos  Neg
0  g o o d   q u a l i t i   d o g   f o o d  good qualiti dog food  ...    1    0
1                            a d v e r t i s               advertis  ...    0    1
2                      d e l i g h t   s a y            delight say  ...    1    0
3                  c o u g h   m e d i c i n          cough medicin  ...    0    1
4                      g r e a t   t a f f i            great taffi  ...    1    0

[5 rows x 5 columns]


In [19]:
data_train, data_test = train_test_split(Data, test_size=0.1, random_state=42)

In [27]:
all_training_words = [word for tokens in data_train['Tokens'] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train['Tokens']]
Training_vocab = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(Training_vocab)))
print("Max sentence length is %s" %max(training_sentence_lengths))

45910 words total, with a vocabulary size of 43
Max sentence length is 83


In [29]:
all_test_words = [word for tokens in data_test['Tokens'] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test['Tokens']]
TEST_VOCAB = sorted(list(set(all_test_words)))
print('%s words total, with a vocabulary size of %s' % (len(all_test_words), len(TEST_VOCAB)))
print('Max sentence length is %s' % max(test_sentence_lengths))

5154 words total, with a vocabulary size of 35
Max sentence length is 64


In [31]:
MAX_LEN = 83

In [35]:
tokenizer = Tokenizer(num_words=len(Training_vocab), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 36 unique tokens.


In [39]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_LEN)
print(train_cnn_data[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  8  4 19  1  2 14  1 16]


In [40]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-11-23 15:01:04--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-23 15:01:04--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-23 15:01:04--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [41]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [42]:
#using glove model for word embeddings
embeddings_index = dict()
f = open('glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [43]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((len(Training_vocab), 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

# print(embedding_matrix[1])

In [61]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_LEN)

In [45]:
#create the model with CNN 5 filters are used and with a hidden layer and non trainable embedding layer
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
  embedding_layer = Embedding(num_words, embedding_dim, weights=[embeddings],
                              input_length=max_sequence_length, trainable=False)
  sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
  embedded_sequences = embedding_layer(sequence_input)

  convs = []
  filter_sizes = [2,3,4,5,6]

  for filter_size in filter_sizes:
    l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
    l_pool = GlobalMaxPooling1D()(l_conv)
    convs.append(l_pool)
  
  l_merge = concatenate(convs, axis=1)
  x = Dropout(0.1)(l_merge)
  x = Dense(128, activation='relu')(x)
  x = Dropout(0.2)(x)
  preds = Dense(labels_index, activation='sigmoid')(x)

  model = Model(sequence_input, preds)
  model.compile(loss='binary_crossentropy', optimizer='adam',
                metrics=['mse'])
  model.summary()
  return model

In [53]:
label_names = ['Pos','Neg']
y_train = data_train[label_names].values
print(y_train)
model = ConvNet(embedding_matrix, 50, len(Training_vocab), 300, len(list(label_names)))
x_train = train_cnn_data

[[1 0]
 [1 0]
 [0 1]
 ...
 [1 0]
 [1 0]
 [0 1]]
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 50, 300)      12900       input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 49, 200)      120200      embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 48, 200)      180200      embedding_3[0][0]                
_______________________________________

In [55]:
num_epochs = 3
batch_size = 34

In [57]:
hist = model.fit(x_train, y_train, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/3
72/72 [==============================] - 1s 11ms/step - loss: 0.4446 - mse: 0.1442 - val_loss: 0.4759 - val_mse: 0.1533
Epoch 2/3
72/72 [==============================] - 1s 9ms/step - loss: 0.4068 - mse: 0.1348 - val_loss: 0.4664 - val_mse: 0.1491
Epoch 3/3
72/72 [==============================] - 1s 9ms/step - loss: 0.3682 - mse: 0.1205 - val_loss: 0.4612 - val_mse: 0.1505


In [62]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

1/1 [==============================] - 0s 7ms/step


In [63]:
labels = [1,0]
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [65]:
sum(data_test.Score==prediction_labels)/len(prediction_labels)

0.7866666666666666

In [66]:
sentence = "worst product ever"
clean_reviewData = []
clean_reviewData.append( " ".join(data_clean(sentence)))

In [67]:
print(clean_reviewData)

['b e s t   p r o d u c t   e v e r']


In [73]:
vocabulary_size = len(Training_vocab)
tokenizer = Tokenizer(num_words=len(Training_vocab), lower=True, char_level=False)
tokenizer.fit_on_texts(clean_reviewData)
training_sequences = tokenizer.texts_to_sequences(clean_reviewData)
training_sequences = pad_sequences(training_sequences, maxlen=MAX_LEN)
print(training_sequences)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  4  1  5  2
   6  3  7  8  9 10  2  1 11  1  3]]


In [91]:
output = model.predict(training_sequences)
# print(output)
print(label_names[np.argmax(output)])
Accuracy = max(output[0])*100
print("Accuracy: %.2f per cent"%Accuracy)

Neg
Accuracy: 51.61 per cent
